# Sentiment Analysis of Yelp Reviews

What the first step you take when you plan to go to a restaurant you have no idea about? You open Yelp! Yelp ratings and reviews help you understand how good a restaurant is. People are so pationate about food that they leave bing long paragrpahs of reviews these days, more so when they don't like the place. Go to yelp page of any restaurant and you can find 100's of reviews. But who has the time to go through all of those review ??? And the 5-point rating system can be very ambigious, a 4 for me can be a 2 for you! So what's the solution ? Woudn't it be great if Yelp added a simple smiley showing happy, neutral or sad face next to review. This will give a  clear picture of whether the review is positive or negative. Since Yelp, has not done it yet, lets learn how we can do it using the art of data science.

# Data Collection

The first step in any data scince task is to gather the data! In order to perform sentiment analysis of Yelp review, we will first need some labelled data, i.e. data which contains the reviews and the related sentiment for those reviews. This can be done in two ways - tkae it of the internet or scrape it from Yelp directly. We will do both! 

For training and testing our model, I will be using the a dataset containing 10,000 review present in the link - https://www.dropbox.com/s/wc6rzl1a2os721d/yelp.csv?dl=0

For running our model to understand the sentiments around a particualr restaurant, I will scrape the Yelp page of that restaurant. This is discused in detail later.

In [122]:
# Import libraries
import pandas as pd
import numpy as np
import nltk
import warnings
warnings.filterwarnings('ignore')

In [123]:
# Load data from csv file
df_yelp_raw_data = pd.read_csv('Yelp_Reviews.csv')

A sample view of the data collected is given below. 

In [124]:
df_yelp_raw_data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


# Data pre-processing / cleaning

Almost always, the data colelcted in not ready-to-eat. We have to pre-process the data to amke it fit for our modelling process.

The 10,000 row dataset we collected contains a lot of intervation about each review. For the pupose of sentiment analysis, we are only interested in the review text and ratings. A sample view of the our relevant data is given below.

In [125]:
#Drop extra columns
df_yelp_review = df_yelp_raw_data[['text' , 'stars']]
df_yelp_review.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


For creating a model which can classify a review as positive or negative, we first need our training data to have the same classsification. This is called labelling the data. 

In this tutorial, I will perform this task in 2 different ways and will compare the results.



### Data pre-processing Approach 1 

For the first approach, I will consider all reviews with rating 1 or 2 as a negative reivew and will label it with a '0' and all reviews with rating 3, 4 or 5 as a positive review and will label it with '1'

In [126]:
#Creating data labels considering rating = 1 or 2 as negative and rest positive.
df_yelp_review['stars'] = df_yelp_review['stars'].apply(lambda x : 1 if x > 3 else 0)

Based on our definition, let's check how many review are positive and negative in our dataset.

In [127]:
df_yelp_review.stars.value_counts()

1    6863
0    3137
Name: stars, dtype: int64

The values above indicate that 6863 reviews are considered as positive review and 3137 reviews are considered as negative review in our dataset.


Now that we have our data labelled, the next step is to clean the text in our data. Oue text contians symbols, puntuations, new line characters, etc. These are not relevant for predicting the sentiment of a review, so I will remove them from the data.

Since cleaning the text is a common activity and has to be done every time we colelct data, so I will make a function for this. The fucntion will use replace fucntion and regex to remove unwanted text. 

In [128]:
# Remove punctutions, new line tag and covert to lower case
def cleanText(df):
    df['text'] = df['text'].str.replace('[^\w\s]','')
    df['text'] = df['text'].str.replace('\n',' ')
    df['text'] = df['text'].apply(lambda x : x.lower())
    return df
    
# df_yelp_review['text'] = df_yelp_review['text'].str.replace('[^\w\s]','')
# df_yelp_review['text'] = df_yelp_review['text'].str.replace('\n',' ')
# df_yelp_review['text'] = df_yelp_review['text'].apply(lambda x : x.lower())

In [129]:
#Call the fuction to clean text in dataset
df_yelp_review = cleanText(df_yelp_review)

Another important part of clening a text data is lemmatization and stopword removal. For lemmatization, we will using NLTK's lemmatize function. 
Stopwords are common words like 'a', 'an', etc. NLTK library has a set of standard stopwords defined, we will use them for removing the stopwords from our text.

In [133]:
# Tokenize, lammatize and remove stopwords

from nltk.corpus import stopwords
import nltk.stem.wordnet as wordnet

stopwrds = set(stopwords.words("english"))

def lemmatiseAndRemoveStopwords(text, lemmatizer=wordnet.WordNetLemmatizer()):
    token = nltk.word_tokenize(text)
    lametized_token = [lemmatizer.lemmatize(t) for t in token]
    for word in lametized_token:
        if word in stopwrds:
            lametized_token.remove(word)
    text = " ".join(lametized_token)
    return text


Since lemmatization and stopword removal has to be done for all texts, I defined a fucntion for it.

In [134]:
df_yelp_review['text'] = df_yelp_review['text'].apply(lambda x : lemmatiseAndRemoveStopwords(x))

# Train and Test data creation

Now that we have clean data ready, we will split it into two sets. One set containing 80% of the data will be used for training the model. Remaining 20% of the data will be used for testing the model. Since the focus of this tutorial is Sensitivity analysis, I am simply using random 80% of the sample for training and remaining 20% for testing. This approach has some drwabacks and a better way of model fitting is cross validation in which you split your data into k folds, fit your model on k-1 folds and test on the kth fold allowing each fold to act as test once. For more information on Cross validation, refer to the link - https://en.wikipedia.org/wiki/Cross-validation_(statistics)

In [146]:
#Split the data into 80-20. 
#We will use 80% of the data for training and 20% for testing

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df_yelp_review, test_size=0.2)

#Reset the index
train_data.reset_index()
test_data.reset_index()

#Convert to a list of strings
train_data_list = train_data.text.tolist()
test_data_list = test_data.text.tolist()

# Feature engineering : Using bag of words technique

The most obvious and common problem in Sentiment Analysis is that the input data is plain text. We cannot just feed raw text in machine learning models. To work with learning models, we have to convert our text data into some sort of mathematical representaion. Bag of words is simple technique to do that. It creates a vocablury from all docuemtns and then counts the number of times each word appears in a document. To create feautures using the bag of words model, I will use the Vectorizer provided in Scikit-learn library.

In [147]:
from sklearn.feature_extraction.text import CountVectorizer

#  Create features from trainign data using Count Vectorizer 
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None) 

train_features = vectorizer.fit_transform(train_data_list)
train_features = train_features.toarray()


In [148]:
#Check how many features are created
train_features.shape

(8000, 29326)

As we can see above, the bag of words model created 28901 features from our training data. We can also check the vocabulory created using the code below.

In [149]:
#To check the vocablury of words created
vocab = vectorizer.get_feature_names()

Bag of words only takens in the counts. To improve upon that, we can also create features using TFIDF which uses the inverse document frequency to contain the effect of highly repeated words.

In [150]:
#TFIDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None) 

train_tfidf = tfidf.fit_transform(train_data_list)
train_tfidf = train_tfidf.toarray()

# Modelling
Using the features created, we will now train a model. There are many machine learning models that can be used for classification. In this tutorial, we will use Random Forest and Multinomial Naive Bayes for model fitting.

## Random Forest
Random forest is a decision tree based ensemble learning model generally used for classification. It works on the idea of decision tree but avoids overfitting by selecting split features from a random set. To learn more about Random Forest , you can have a look the link - https://en.wikipedia.org/wiki/Random_forest. We will use the Random Forest classifier provided by scikit- learn library.

In [151]:
# Import Random Forest classifier from scikit learn
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators = 100) 
# Fit the forest using bag of words features 
rf = rf.fit( train_features, train_data["stars"] )

We will now test our model. To do that, we will first create bag of words feature for the test data and than predict the sentiment using our model.

In [152]:
# Create bag of words for the test data
test_features = vectorizer.transform(test_data_list)
test_features = test_features.toarray()

# Use the random forest to make sentiment label predictions
rf_pred = rf.predict(test_features)

In [153]:
# Confusion matrix for the prediction
from sklearn.metrics import confusion_matrix

y_true = test_data.stars.tolist()
y_pred = rf_pred
confusion_matrix(y_true, y_pred)

array([[ 259,  346],
       [  44, 1351]])

In [154]:
# Accuracy of the model
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.80500000000000005

The random forest model fit using abg of words features gives an accuracy of around 80%.

We can also fit our forest model using the TFIDF feature created. 

In [155]:
#Using tfidf
rf_tfidf = RandomForestClassifier(n_estimators = 100)
rf_tfidf = rf_tfidf.fit(train_tfidf, train_data["stars"])

test_tfidf = tfidf.transform(test_data_list)
test_tfidf = test_tfidf.toarray()

result_tfidf = rf_tfidf.predict(test_tfidf)

In [156]:
# Confusion matrix with TFIDF features
confusion_matrix(y_true, result_tfidf)
accuracy_score(y_true, result_tfidf)

0.79200000000000004

The Random forest model fit using TFIDF feature also gives an accuracy of around 80%.

## Multinomial Naive Bayes
The multinomial naive bayes model is a version of naive bayes that performs better on text documents. We will use our bag of words features to fit and predict using this model.

In [157]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb = nb.fit(train_features, train_data["stars"])

In [158]:
preds = nb.predict(test_features)

In [159]:
y_true = test_data.stars.tolist()
y_pred = preds
confusion_matrix(y_true, y_pred)
accuracy_score(y_true, y_pred)

0.83150000000000002

The multinomial naive bayes is giving an accuracy of aroud 83%.

### Data pre-processing Approach 2
In the previous approach, we were considering all reviews with rating 1 and 2 as negative and rest as positive. Using this approach, the models we created gave accuracy in the lower 80's. Now we take a more conservative appraoch and will consider only extreme reviews.

In this appraoch, we will consider only review with 1 rating as negaitve and reviews with 5 rating as positive. We will ignore all other reviews while training our model.

In [165]:
df_raw_data = pd.read_csv('Yelp_Reviews.csv')
data_extreme = df_raw_data[(df_raw_data['stars'] == 1) | (df_raw_data['stars'] == 5)]
data_extreme['stars'] = data_extreme['stars'].apply(lambda x : 1 if x == 5 else 0)
data_extreme.stars.value_counts()

1    3337
0     749
Name: stars, dtype: int64

Using this approch, we have 3337 review as positive and 749 as negative. Now we will clean the data as before, split the data into test and train and create bag of words features for modelling. 

In [166]:
# Data cleaning 
data_extreme = cleanText(data_extreme)
data_extreme['text'] = data_extreme['text'].apply(lambda x : lemmatiseAndRemoveStopwords(x))

#Split the data into 80-20. 
#We will use 80% of the data for training and 30% for testing
train_data_ext, test_data_ext = train_test_split(data_extreme, test_size=0.2)

#Reset the index
train_data_ext.reset_index()
test_data_ext.reset_index()

# Convert to list
train_ext_list = train_data_ext.text.tolist()
test_ext_list = test_data_ext.text.tolist()




In [169]:
# Create bag of word vector
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None) 

train_features_ext = vectorizer.fit_transform(train_ext_list)
#Convert to numpy array
train_features_ext = train_features_ext.toarray()

### Random Forest for 2nd approach

In [170]:
# Fit the random forest classifier
rf_ext = RandomForestClassifier(n_estimators = 100) 
rf_ext = rf_ext.fit( train_features_ext, train_data_ext["stars"] )

In [171]:
# Create bag of words vector for test data
test_features_ext = vectorizer.transform(test_ext_list)
test_features_ext = test_features_ext.toarray()

# Predict using above model
result_ext = rf_ext.predict(test_features_ext)
y_true = test_data_ext.stars.tolist()
y_pred = result_ext

In [172]:
# Confusion matrix for random forest using 2nd approach
confusion_matrix(y_true, y_pred)

array([[ 48, 117],
       [  2, 651]])

In [173]:
# Accuracy for random forest fit using 2nd approach
accuracy_score(y_true, y_pred)

0.8545232273838631

The random forest model fit using this approach gives an accuracy of around 85%. This shows the importance of data preprocessing and feature creation in data science. 

### Multinomial Naive Bayes for 2nd approach

In [176]:
# Fit the multinomial naive bayes model
nb_ext = MultinomialNB()
nb_ext = nb_ext.fit(train_features_ext, train_data_ext["stars"])

#Predict of test data using this model
preds_ext = nb_ext.predict(test_features_ext)
y_true = test_data_ext.stars.tolist()
y_pred = preds_ext

In [177]:
# Confusion matrix for naive bayes using 2nd approach
confusion_matrix(y_true, y_pred)

array([[108,  57],
       [ 12, 641]])

In [178]:
# Accuracy for naive bayes using 2nd approach
accuracy_score(y_true, y_pred)

0.91564792176039123

The multinomial naive bayes model fit using 2nd approach gives an accuracy of around 92%. This is a significant improvement but this is only based on training and test data. The real performance of any model is judged by how is works on completely unseen data. This is what we will check in the next half of this tutorial.

# Deriving public sentiment for a restaurant

Ratings of a restaurant is good overall representaion of public opinion. But, rating can be done only on 1-5 scale. People write a lot more in reviews and can be a better judge on public sentiment. We will now check that using our model. 

To check public opinion for a restaurant using sentoment analysis, we first need the reviews for that restaurant. As mentioned initially in this tutorial, scrapping is a technique using which you can collect information from Yelp page of any restaurant. Using the code below, we hit the Yelp page of cafe in San Francisco called The Temporarium Coffee And Tea, scrape all reviews and put them in dataframe. Then, we use our model to predict whether people are talking positively or negatively about it.

In [84]:
import requests
from bs4 import BeautifulSoup

def retrieve_html(url):
    
    response = requests.get(url)
    return(response.status_code, response.text)

def parse_page(html):

    soup = BeautifulSoup(html, 'html.parser') 
    review_list = []

    for review in soup.find_all("div", class_="review review--with-sidebar"):
        rt = review.find("div", class_="rating-large")
        rating = rt['title']
        text = review.find("p")
        
        review_dict = {'stars':float(rating[:3]),'text': text.text.strip()}
        review_list  = review_list + [review_dict]
    
    url = None
    current_page = soup.find("span", class_="pagination-links_anchor")
    if current_page is not None:
        current_page = int(current_page.text.strip())
    for page in soup.find_all("a", class_="available-number pagination-links_anchor"):
        if page is not None:            
            page_num = int(page.text.strip())            
            if page_num == current_page+1:
                url = page['href']   
    return (review_list, url)

def extract_reviews(url):
    
    review_list = []
    while url != None:
        yelp_html = retrieve_html(url)
        r_list , url = parse_page(yelp_html[1])
        review_list += r_list
    return review_list
    pass

data = extract_reviews('https://www.yelp.com/biz/the-temporarium-coffee-and-tea-san-francisco')
df = pd.DataFrame()
df = df.append(data, ignore_index=True)

Once we have all the reviews for this restaurant, we will follow the same cleaning and preprossing we did during our model fitting. We then create bag of words vector for this data.

In [183]:

# Data cleaning 
df = cleanText(df)

#Clean this data
# df['text'] = df['text'].str.replace('[^\w\s]','')
# df['text'] = df['text'].str.replace('\n',' ')
# df['text'] = df['text'].apply(lambda x : x.lower())
df['text'] = df['text'].apply(lambda x : lemmatiseAndRemoveStopwords(x))
#Convert to list
validation_list = df.text.tolist()
# Create bag of words features
features = vectorizer.transform(validation_list)
features = features.toarray()


### Analyzing public sentiment using the multinomial naive bayes model

In [186]:
validation_predict_nb = nb_ext.predict(features)
validation_predict_nb.sum()
percent_positive = (validation_predict_nb.sum() / len(validation_predict_nb))*100
percent_positive

98.701298701298697

The model suggests that almost 99% of the reviews for this restaurant are positive. This matches with the overall '5' rating of this restaurant on Yelp. But the interesthing thing to see is the model found some negative review for this famous restaurrant. Now, catching those reviews among 100's of review direclty on Yelp would be difficult. This is one way how sentiment analysis can be use. 

Let's check which review was predicted as negative by our model.

In [193]:
data[validation_predict_nb.tolist().index(0)]['text']

"I had an odd interaction with the guy working there. He was busy talking to a customer/friend and didn't realize I wanted something. Anyway, the ice coffee was okay but overpriced and probably unsanitary (guy pulled it from a shelf and spilled half of it on the ground). I can't say that this was a comfortable coffee shop experience."

What we saw in this tutorial is a simple example of how and what can be done using sentiment analysis. Analyzing text is very interesting field and if you are interested in going deep into it, you can check deep learning models. Overall, sentiment analysis can be very useful in understanding overall sentiment of 1000's of reviews. 

# References
1. https://medium.com/tensorist/classifying-yelp-reviews-using-nltk-and-scikit-learn-c58e71e962d9
2. https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words